In [ ]:
#######################################################
#######################################################
############    COPYRIGHT - DATA SOCIETY   ############
#######################################################
#######################################################

## INTRO TO TEXT MINING PART 4 ##

## NOTE: To run individual pieces of code, select the line of code and
##       press ctrl + enter for PCs or command + enter for Macs




In [ ]:
#=================================================-
#### Slide 3: Import packages  ####

# Helper packages.
from pathlib import Path
import os
import pickle
import pandas as pd
import numpy as np

# Cosine similarity and clustering packages.
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import ward, dendrogram, fcluster
from gensim import matutils
# Network creation and visualization.
import networkx as nx
from pyvis.network import Network

# Other plotting tools.
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt





In [ ]:
#=================================================-
#### Slide 4: Directory settings  ####

# Set 'main_dir' to location of the project folder
home_dir = Path(".").resolve()
main_dir = home_dir.parent
print(main_dir)
data_dir = str(main_dir) + "/data"
plot_dir = str(main_dir) + "/plots"
print(data_dir)
os.chdir(data_dir)




In [ ]:
#=================================================-
#### Slide 5: Load objects   ####

processed_docs = pickle.load(open("NYT_clean.sav","rb"))  #<- the processed NYT snippets
corpus_tfidf = pickle.load(open("corpus_tfidf.sav","rb"))
bow_corpus = pickle.load(open("bow_corpus.sav","rb"))
dictionary = pickle.load(open("dictionary.sav","rb"))
DTM = pickle.load(open("DTM.sav","rb"))
doc_topic_pairs_df = pickle.load(open("doc_topic_pairs_df.sav","rb"))

# Load NYT article data from original file.
NYT = pd.read_csv('NYT_article_data.csv')

# Load pre-saved word counts array we pickled.
word_counts_array = pickle.load(open("word_counts_array.sav","rb"))




In [ ]:
#=================================================-
#### Slide 13: Compute TF-IDF weighted TDM (cont'd)  ####

# Convert corpus weighted with TF-IDF to a TDM matrix.
TDM_tf_idf_matrix = matutils.corpus2dense(corpus_tfidf,
                                          DTM.shape[1],
                                          DTM.shape[0])
print(TDM_tf_idf_matrix.shape)
print(type(TDM_tf_idf_matrix))




In [ ]:
#=================================================-
#### Slide 14: Transpose TDM to DTM  ####

# Transpose matrix to get the DTM.
DTM_tf_idf_matrix = TDM_tf_idf_matrix.transpose()
print(DTM_tf_idf_matrix.shape)




In [ ]:
#=================================================-
#### Slide 15: Convert it to DTM and create a dataframe  ####

# Create the DTM weighted with TF-IDF.
valid_snippets = np.where(word_counts_array >= 5)[0]

DTM_tf_idf = pd.DataFrame(DTM_tf_idf_matrix, 
                          columns = DTM.columns,  #<- set column names to those in DTM
                          index = valid_snippets) #<- set index to original article index
print(DTM_tf_idf.head())




In [ ]:
#=================================================-
#### Slide 16: Compute cosine similarity matrix  ####

# Compute similarity matrix (a numpy 2D array).
similarity = cosine_similarity(DTM_tf_idf_matrix)
print(type(similarity))
print(similarity.shape)




In [ ]:
#=================================================-
#### Slide 19: Save similarity data as a dataframe  ####

# Create similarity dataframe with appropriate column names and indices.
similarity_df = pd.DataFrame(similarity,
                             columns = valid_snippets,
                             index = valid_snippets)
                             




In [ ]:
#=================================================-
#### Slide 21: Exercise 1  ####






In [ ]:
#=================================================-
#### Slide 24: Compute a graph from similarity object  ####

# Create a graph object from the similarity matrix.
graph = nx.from_pandas_adjacency(similarity_df)





In [ ]:
#=================================================-
#### Slide 26: Compute an edgelist from graph  ####

# Convert it to a dataframe in a form of an edgelist.
edgelist_df = nx.to_pandas_edgelist(graph)

# Take a look at the dataframe of edges.
print(edgelist_df.head())
print(edgelist_df.shape)




In [ ]:
#=================================================-
#### Slide 27: Cosine similarity score distribution  ####

# Plot the weights of edges (i.e. similarity scores).
plt.clf()
plt.rcParams.update({'font.size': 10})
plt.hist(edgelist_df['weight'])
plt.xlabel('Cosine similarity score') 
plt.title('Cosine similarity score distribution')
plt.show()




In [ ]:
#=================================================-
#### Slide 28: Filter out edges  ####

# Filter out all entries below 0.4 and above 0.9.
edgelist_df = edgelist_df.query('weight>0.4 and weight<0.9')
# Plot the weights of edges (i.e. similarity scores).
plt.hist(edgelist_df['weight'])
plt.xlabel('Cosine similarity score') 
plt.ylabel('Score counts')  
plt.title('Cosine similarity score distribution')
plt.show()

plt.clf()
# Plot the weights of edges (i.e. similarity scores).
plt.hist(edgelist_df['weight'])
plt.xlabel('Cosine similarity score') 
plt.ylabel('Score counts')  
plt.title('Cosine similarity score distribution')
plt.show()




In [ ]:
#=================================================-
#### Slide 29: Check edges data and number of edges  ####

# Take a look at the dataframe of edges.
print(edgelist_df.head())
print(edgelist_df.shape)




In [ ]:
#=================================================-
#### Slide 32: Create network and set base parameters (cont'd)  ####

# Create an empty network object.
network = Network(height="100%", 
                  width="60%", 
                  bgcolor="#FFFFFF", 
                  font_color="#000000")

# Set the physics layout of the network.
network.force_atlas_2based()
network.set_edge_smooth('dynamic')
print(network)




In [ ]:
#=================================================-
#### Slide 33: Populate network with edge and node data  ####

# Zip columns of edgelist data into a set of tuples.
edge_data = zip(edgelist_df['source'], edgelist_df['target'], edgelist_df['weight'])

# Iterate through the edge data.
for e in edge_data:
    src = e[0] #<- get the source node
    dst = e[1] #<- get the destination (i.e. target node)
    w = e[2]   #<- get the weight of the edge

    # Add a source node with its information.
    network.add_node(src, src, title = src) 
    # Add a destination node with its information.
    network.add_node(dst, dst, title = dst)
    # Add an edge between source and destination nodes with weight w.
    network.add_edge(src, dst, value = w)





In [ ]:
#=================================================-
#### Slide 34: Inspect populated network  ####

print(network.nodes[0:5])
print(network.edges[0:5])




In [ ]:
#=================================================-
#### Slide 35: Get neighbor map for each node  ####

# Get a list of node neighbors.
neighbor_map = network.get_adj_list()

# Show documents most similar to document 17.
print(neighbor_map[17])




In [ ]:
#=================================================-
#### Slide 36: Add that information into the hover over  ####

# Add neighbor data to node hover data.
for node in network.nodes:
    title = "Most similar articles: <br>"
    neighbors = list(neighbor_map[node["id"]])
    title = title + "<br>".join(str(neighbor) for neighbor in neighbors)
    node["title"] = title

print(network.nodes[0])




In [ ]:
#=================================================-
#### Slide 37: Save and show the network graph  ####

# Save html and show graph in browser.
network.show(plot_dir + "/NYT_similar_snippets.html")




In [ ]:
#=================================================-
#### Slide 40: What does "most similar articles" mean?  ####

edgelist_df_subset = edgelist_df.query("source==31")
print(edgelist_df_subset)




In [ ]:
#=================================================-
#### Slide 41: Look up the similar article pairs  ####

print(edgelist_df_subset)
print(NYT.iloc[31, 2])
print(NYT.iloc[118, 2])
print(NYT.iloc[223, 2])




In [ ]:
#=================================================-
#### Slide 42: Show buttons to modify graph look  ####

# Show buttons to modify the look.
network.show_buttons(filter_=['physics'])

# Save html and show graph in browser.
network.show(plot_dir+"/NYT_similar_snippets.html")




In [ ]:
#=================================================-
#### Slide 45: Exercise 2  ####






In [ ]:
#=================================================-
#### Slide 50: Hierarchical clustering: compute distance  ####

# Compute distance matrix by subtracting similarity from 1.
distance = 1 - similarity




In [ ]:
#=================================================-
#### Slide 56: Compute and interpret linkage matrix  ####

# Define the `linkage_matrix` using 
# `ward` clustering algorithm.
linkage_matrix = ward(distance) 
print(linkage_matrix[0:10])




In [ ]:
#=================================================-
#### Slide 57: Compute and interpret linkage matrix (cont'd)  ####

# Print shape of the matrix.
print(linkage_matrix.shape)




In [ ]:
#=================================================-
#### Slide 59: Compute and interpret linkage matrix (cont'd)  ####

print(linkage_matrix[150])




In [ ]:
#=================================================-
#### Slide 60: Compute and interpret linkage matrix (cont'd)  ####

print(linkage_matrix[246])




In [ ]:
#=================================================-
#### Slide 61: Hierarchical clustering: visualize results  ####

# Now we can plot the hierarchical clusters.
fig, axes = plt.subplots(figsize = (15, 40)) 
axes = dendrogram(linkage_matrix, 
                  orientation = "right", 
                  labels = valid_snippets,
                  leaf_font_size = 11)




In [ ]:
#=================================================-
#### Slide 63: Hierarchical clustering: get cluster labels  ####

# Set k - the max number of clusters.
k = 5 
# Get cluster labels for each snippet.
cluster_labels = fcluster(linkage_matrix,         #<- linkage matrix
                          k,                      #<- max number of clusters
                          criterion = 'maxclust') #<- criterion maxclust
print(cluster_labels)




In [ ]:
#=================================================-
#### Slide 64: Combine data with hclust and LDA cluster labels  ####

NYT_valid_articles = NYT.loc[valid_snippets]
NYT_valid_articles['hclust_label'] = cluster_labels
doc_topic_pairs_df = doc_topic_pairs_df.sort_values(by = "doc_id")
NYT_valid_articles['LDA_topic_label'] = doc_topic_pairs_df['best_topic']




In [ ]:
#=================================================-
#### Slide 65: Save plots and data  ####

fig.savefig(plot_dir + '/NYT_hclust.png')
NYT_valid_articles.to_csv(data_dir + '/NYT_snippets_with_cluster_labels.csv')


